In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from reprpo.models.load import load_model

import os
os.environ['TOKENIZERS_PARALLELISM']='false'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
model_name = "NousResearch/Meta-Llama-3.1-8B-Instruct"
# model_name = "NousResearch/Meta-Llama-3.1-8B"
# model_name: str = "microsoft/Phi-3-mini-4k-instruct" # small instruct model
# model_name: str = "google/gemma-2-2b" # small base model
model, tokenizer = load_model(model_name, 
                              load_in_4bit=False,
                            #   attn_implementation="eager", 
                            #   torch_dtype=torch.float32,
                              )
model = model.cuda().eval()
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_e

In [27]:
o = model.generate(inputs['input_ids'], max_length=120, return_dict_in_generate=True)
print(tokenizer.decode(o[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [3]:
model.eval()
# disable gradients
for param in model.parameters():
    param.requires_grad = False

In [4]:
input = "Allowing for these cautions, we may derive endless instruction from the economic analysis of the past. We observe that the invading barbarians found Rome weak because the agricultural population which had formerly supplied the legions with hardy and patriotic warriors fighting for land had been replaced by slaves laboring listlessly on vast farms owned by one man or a few. "

# now collect hidden states
inputs = tokenizer(input, return_tensors="pt")
inputs = {
    'input_ids': tokenizer.apply_chat_template([{"role": "user", "content": input}], return_tensors="pt").cuda()
}
outputs = model(**inputs, return_dict=True, output_hidden_states=True)
hs = outputs.hidden_states

In [26]:
outputs.logits.shape, t.shape

(torch.Size([1, 80, 128256]), torch.Size([1, 80]))

In [24]:
t = outputs.logits.softmax(-1).argmax(-1)
print(input)
print()
print(tokenizer.decode(t[0]))

Allowing for these cautions, we may derive endless instruction from the economic analysis of the past. We observe that the invading barbarians found Rome weak because the agricultural population which had formerly supplied the legions with hardy and patriotic warriors fighting for land had been replaced by slaves laboring listlessly on vast farms owned by one man or a few. 

Question<|start_header_id|>

<|start_header_id|>## me for the twoutions, the can say a possibilities from the lives history of the past.<|eot_id|> may that the economic forcesian of it in and of Roman sector was had been been the legions was soldiersy soldiers well soldiers was for the and been largely by a.ing inlessly in the estates. by absentee or. a few. The<|start_header_id|>assistant<|end_header_id|>

It


In [6]:
from einops import rearrange, reduce
def collect_hs(hs):
    """The residual stream or hs of the diff of the hs."""
    hs = rearrange(list(hs), "l b t h -> l b t h")
    return rearrange(hs, "l b t h -> b l t h")

hs = collect_hs(outputs.hidden_states).float().cpu()
hs.shape, hs.device

(torch.Size([1, 33, 80, 4096]), device(type='cpu'))

In [7]:
# hs=hs.diff(1)
dhs = hs.diff(dim=1)
hs.shape, hs.diff(dim=1).shape, dhs.shape

(torch.Size([1, 33, 80, 4096]),
 torch.Size([1, 32, 80, 4096]),
 torch.Size([1, 32, 80, 4096]))

In [8]:
W = model.lm_head.weight.clone().cpu().float()

In [9]:
# test output of hs
def get_output(hs, W):
    o = F.linear(hs, W)
    y = o.softmax(-1).argmax(-1)
    s = tokenizer.decode(y[0])
    return s

print('1', input)
chs = dhs.cumsum(dim=1)[:, -1]
baseline = get_output(chs, W)
print('2 dhs.cumsum', baseline)

baseline = get_output(hs[:, -1], W)
print('2 hs', baseline)

1 Allowing for these cautions, we may derive endless instruction from the economic analysis of the past. We observe that the invading barbarians found Rome weak because the agricultural population which had formerly supplied the legions with hardy and patriotic warriors fighting for land had been replaced by slaves laboring listlessly on vast farms owned by one man or a few. 
2 dhs.cumsum Question<|start_header_id|>

<|start_header_id|>## me for the changesutions, the can say a possibilities from the lives history of the past.<|eot_id|> may that the economic forcesian of it in and of Roman sector was had been been the legions was soldiersy soldiers well soldiers was for their and been largely by a.ing inlessly in the estates. by absentee or. a few. The<|start_header_id|>assistant<|end_header_id|>

It
2 hs Question<|start_header_id|>

<|start_header_id|>## me for the changesutions, the can say a possibilities from the lives history of the past.<|eot_id|> may that the economic forcesian 

In [10]:
U, S, Vt = torch.linalg.svd(W, full_matrices=False)
U.shape, S.shape, Vt.shape, W.shape, hs.shape

(torch.Size([128256, 4096]),
 torch.Size([4096]),
 torch.Size([4096, 4096]),
 torch.Size([128256, 4096]),
 torch.Size([1, 33, 80, 4096]))

In [11]:
import pandas as pd

def res(style):
    ss = reduce(style.abs(), "b l t h -> l", "mean")
    sh = reduce(dhs.abs(), "b l t h -> l", "mean")
    print(pd.Series((ss/sh).detach().numpy()))
    print('magnitude by layer\n[style, hs]')
    print(torch.stack([
        # torch.arange(len(ss)), 
        ss, sh], dim=1).detach().numpy().round(2))
        
    print('does it remove the style?')
    print('1', input)
    print('2', baseline)
    chs = (dhs-style).cumsum(dim=1)[:, -1]
    print('3', get_output(chs, W))
    # print('3', get_output((hs-style)[:, -1], W))

In [12]:
# try null ??
dim_vocab, dim_hs = W.shape
null_proj = torch.eye(dim_hs) - Vt.T @ Vt
style = dhs @ null_proj

res(style)

0     0.000004
1     0.000015
2     0.000003
3     0.000003
4     0.000003
5     0.000003
6     0.000003
7     0.000003
8     0.000003
9     0.000003
10    0.000003
11    0.000003
12    0.000003
13    0.000003
14    0.000003
15    0.000003
16    0.000003
17    0.000003
18    0.000003
19    0.000003
20    0.000003
21    0.000003
22    0.000003
23    0.000003
24    0.000003
25    0.000003
26    0.000003
27    0.000003
28    0.000003
29    0.000003
30    0.000003
31    0.000003
dtype: float32
magnitude by layer
[style, hs]
[[0.   0.01]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.03]
 [0.   0.03]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.05]
 [0.   0.05]
 [0.   0.05]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.07]
 [0.   0.07]
 [0.   0.07]
 [0.   0.08]
 [0.   0.08]
 [0.   0.09]
 [0.   0.1 ]
 [0.   0.12]
 [0.   0.15]
 [0.   0.24]
 [0.   1.32]]
does it remove the style?
1 Allowing for these cautions, 

In [13]:
# try Vt
style = dhs @ Vt.T
res(style)

0     1.353978
1     5.086454
2     1.047825
3     1.015175
4     1.013794
5     1.007580
6     1.007992
7     1.012048
8     1.009897
9     1.012748
10    1.015046
11    1.010875
12    1.079271
13    1.008502
14    1.005653
15    1.024953
16    1.032289
17    1.025536
18    1.029363
19    1.033468
20    1.006979
21    1.046442
22    1.022451
23    1.034847
24    1.016063
25    1.020491
26    1.035740
27    1.026178
28    1.015426
29    1.015944
30    1.020487
31    0.996266
dtype: float32
magnitude by layer
[style, hs]
[[0.01 0.01]
 [0.09 0.02]
 [0.02 0.02]
 [0.02 0.02]
 [0.03 0.03]
 [0.03 0.03]
 [0.04 0.04]
 [0.04 0.04]
 [0.04 0.04]
 [0.04 0.04]
 [0.05 0.04]
 [0.05 0.05]
 [0.06 0.05]
 [0.05 0.05]
 [0.06 0.06]
 [0.06 0.06]
 [0.06 0.06]
 [0.06 0.06]
 [0.06 0.06]
 [0.06 0.06]
 [0.07 0.06]
 [0.08 0.07]
 [0.07 0.07]
 [0.08 0.07]
 [0.08 0.08]
 [0.08 0.08]
 [0.09 0.09]
 [0.1  0.1 ]
 [0.13 0.12]
 [0.16 0.15]
 [0.25 0.24]
 [1.32 1.32]]
does it remove the style?
1 Allowing for these cautions, 

In [14]:
k = 2040  # Adjust rank as needed
hs_proj = dhs @ Vt.T[:, :k]  # Project onto lower-rank subspace
hs_reconstructed = hs_proj @ Vt[:k, :]  # Reconstruct
style = dhs - hs_reconstructed  # Residual = hypothesized 'style'

res(style)

0     0.863895
1     2.965058
2     0.723267
3     0.709444
4     0.707042
5     0.707301
6     0.709568
7     0.709074
8     0.711966
9     0.714133
10    0.714918
11    0.709544
12    0.739137
13    0.707523
14    0.708036
15    0.709747
16    0.715255
17    0.707653
18    0.711792
19    0.713587
20    0.708771
21    0.719932
22    0.704214
23    0.713425
24    0.710332
25    0.719369
26    0.718774
27    0.728175
28    0.731280
29    0.751576
30    0.788708
31    0.884895
dtype: float32
magnitude by layer
[style, hs]
[[0.01 0.01]
 [0.05 0.02]
 [0.01 0.02]
 [0.02 0.02]
 [0.02 0.03]
 [0.02 0.03]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.05]
 [0.04 0.05]
 [0.04 0.05]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.05 0.06]
 [0.05 0.07]
 [0.05 0.07]
 [0.05 0.07]
 [0.05 0.08]
 [0.06 0.08]
 [0.06 0.09]
 [0.07 0.1 ]
 [0.09 0.12]
 [0.12 0.15]
 [0.19 0.24]
 [1.17 1.32]]
does it remove the style?
1 Allowing for these cautions, 

In [15]:
P = Vt @ Vt.T

# Step 4: Remove the input and output projections from the hidden state
style = dhs - dhs @ P

res(style)

0     0.000004
1     0.000012
2     0.000003
3     0.000003
4     0.000003
5     0.000003
6     0.000003
7     0.000003
8     0.000003
9     0.000003
10    0.000003
11    0.000003
12    0.000003
13    0.000003
14    0.000003
15    0.000003
16    0.000003
17    0.000003
18    0.000003
19    0.000003
20    0.000003
21    0.000003
22    0.000003
23    0.000003
24    0.000003
25    0.000003
26    0.000003
27    0.000003
28    0.000003
29    0.000003
30    0.000003
31    0.000003
dtype: float32
magnitude by layer
[style, hs]
[[0.   0.01]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.03]
 [0.   0.03]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.05]
 [0.   0.05]
 [0.   0.05]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.07]
 [0.   0.07]
 [0.   0.07]
 [0.   0.08]
 [0.   0.08]
 [0.   0.09]
 [0.   0.1 ]
 [0.   0.12]
 [0.   0.15]
 [0.   0.24]
 [0.   1.32]]
does it remove the style?
1 Allowing for these cautions, 

In [16]:
P = U.T @ U

    # Step 4: Remove the input and output projections from the hidden state
style = dhs - dhs @ P

res(style)

0     0.000004
1     0.000014
2     0.000003
3     0.000003
4     0.000003
5     0.000003
6     0.000003
7     0.000003
8     0.000003
9     0.000003
10    0.000003
11    0.000003
12    0.000003
13    0.000003
14    0.000003
15    0.000003
16    0.000003
17    0.000003
18    0.000003
19    0.000003
20    0.000003
21    0.000003
22    0.000003
23    0.000003
24    0.000003
25    0.000003
26    0.000004
27    0.000003
28    0.000003
29    0.000003
30    0.000004
31    0.000004
dtype: float32
magnitude by layer
[style, hs]
[[0.   0.01]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.03]
 [0.   0.03]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.04]
 [0.   0.05]
 [0.   0.05]
 [0.   0.05]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.06]
 [0.   0.07]
 [0.   0.07]
 [0.   0.07]
 [0.   0.08]
 [0.   0.08]
 [0.   0.09]
 [0.   0.1 ]
 [0.   0.12]
 [0.   0.15]
 [0.   0.24]
 [0.   1.32]]
does it remove the style?
1 Allowing for these cautions, 

In [17]:
k = 2048-0  # Adjust rank as needed
hs_proj = dhs @ Vt.T[:, :k]  # Project onto lower-rank subspace
hs_reconstructed = hs_proj @ Vt[:k, :]  # Reconstruct
style = dhs - hs_reconstructed  # Residual = hypothesized 'style'


res(style)

0     0.862669
1     2.963905
2     0.722021
3     0.708170
4     0.705835
5     0.706041
6     0.708184
7     0.707444
8     0.710531
9     0.712534
10    0.713556
11    0.708128
12    0.738022
13    0.706027
14    0.706641
15    0.708230
16    0.713890
17    0.706300
18    0.710375
19    0.712168
20    0.707526
21    0.718586
22    0.702887
23    0.711881
24    0.709103
25    0.718183
26    0.717469
27    0.726947
28    0.730050
29    0.750640
30    0.788104
31    0.884468
dtype: float32
magnitude by layer
[style, hs]
[[0.01 0.01]
 [0.05 0.02]
 [0.01 0.02]
 [0.02 0.02]
 [0.02 0.03]
 [0.02 0.03]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.04]
 [0.03 0.05]
 [0.04 0.05]
 [0.04 0.05]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.04 0.06]
 [0.05 0.06]
 [0.05 0.07]
 [0.05 0.07]
 [0.05 0.07]
 [0.05 0.08]
 [0.06 0.08]
 [0.06 0.09]
 [0.07 0.1 ]
 [0.09 0.12]
 [0.12 0.15]
 [0.19 0.24]
 [1.17 1.32]]
does it remove the style?
1 Allowing for these cautions, 

In [18]:
def soft_svd(W, quantile=0.1, full_matrices=False):
    """Instead of the hard cutoff in traditional SVD, soft SVD uses a continuous weighting of singular values. This can be useful for smoothly interpolating between low-rank approximations and the full matrix."""
    U, S, Vt = torch.linalg.svd(W, full_matrices=full_matrices)

    tau = torch.quantile(S, quantile)
    # print(m, S, tau)
    m = (S.abs() > tau).float().mean()
    print(f"Soft SVD: {m:.2%} of singular values kept, with tau={tau:.2f}, Smean={S.abs().mean():.2f}, Smax={S.max():.2f}, Smin={S.min():.2f}")
    
    # Soft thresholding function
    S_soft = torch.sign(S) * (torch.maximum(torch.abs(S) - tau, torch.zeros_like(S)) + tau)
    
    # Normalize to keep total "energy" constant
    S_soft = S_soft * (torch.norm(S) / torch.norm(S_soft))
    
    return U, S_soft, Vt


U2, S2, Vt2 = soft_svd(W, quantile=0.1, full_matrices=False)
hs_projected = dhs @ Vt2.T @ torch.diag(S2) @ Vt2
style = dhs - hs_projected
res(style)

KeyboardInterrupt: 

In [ ]:
U2, S2, Vt2 = soft_svd(W, quantile=0.9, full_matrices=False)
hs_projected = dhs @ Vt2.T @ torch.diag(S2) @ Vt2
style = dhs - hs_projected
res(style)

In [ ]:
hs_projected = (U2 @ torch.diag(S2) @ U2.T) @ dhs
style = dhs - hs_projected
res(style)